GlobalWoZ là dataset task-oriented dialogue (TOD) kiểu MultiWOZ: File JSON với 11.5 triệu dòng dữ liệu hội thoại Tiếng Việt nhiều lượt (multi-turn)

Nhiều domain (hotel, restaurant, taxi, attraction, …)

Mục tiêu là: giúp user hoàn thành task (tìm khách sạn + đặt phòng + tìm nhà hàng + đặt bàn…)

File là một dict: dialog_id -> dialogue_object

Số lượng hội thoại: 10437 dialogues

Mỗi dialogue chỉ có 2 trường:

goal: mục tiêu người dùng cần đạt

log: toàn bộ các lượt nói (turns) của user và system

Nghĩa là dữ liệu này là “hội thoại hoàn chỉnh”, bạn có thể dùng để:

train chatbot kiểu task-oriented (end-to-end)

hoặc dùng làm “bộ test” để đánh giá chatbot/LLM

In [ ]:
!python inspect_f2e_vi.py

Top-level type: <class 'dict'>
Num dialogs: 10437
First dialog id: MUL0003.json
Fields in one dialog: ['goal', 'log']

--- preview ---
{
  "goal": {
    "attraction": {},
    "hospital": {},
    "hotel": {
      "book": {
        "day": "sunday",
        "invalid": false,
        "people": "6",
        "stay": "4"
      },
      "fail_book": {},
      "fail_info": {},
      "info": {
        "internet": "yes",
        "parking": "yes",
        "pricerange": "cheap",
        "type": "guesthouse"
      }
    },
    "police": {},
    "restaurant": {
      "book": {
        "day": "sunday",
        "invalid": false,
        "people": "6",
        "time": "18:45"
      },
      "fail_book": {},
      "fail_info": {},
      "info": {
        "area": "centre",
        "food": "italian",
        "pricerange": "cheap"
      }
    },
    "taxi": {},
    "train": {}
  },
  "log": [
    {
      "dialog_act": {
        "Hotel-Inform": [
          [
            "internet",
            "yes"
        

In [1]:
!python export_dialogue_for_slide.py

Saved: demo_globalwoz_vi_slide.md and demo_globalwoz_vi_one_dialogue.json
Open demo_globalwoz_vi_slide.md để copy vào slide notes.


In [2]:
!python find_multidomain_dialogue.py

Top candidates (>=2 domains):
2 MUL0003.json
2 MUL0004.json
3 MUL0011.json
2 MUL0014.json
3 MUL0018.json


In [14]:
!python print_one_dialogue.py


Dialog ID: MUL0003.json
=== GOAL (tóm tắt domain có yêu cầu) ===
Domains: ['hotel', 'restaurant']

=== TURNS ===
[00] USER: tôi đang tìm một nơi để ở. nó phải là guesthouse và có wifi miễn phí .
[01] SYSTEM: có 23 hotels đáp ứng nhu cầu của bạn . bạn có muốn thu hẹp tìm kiếm của mình theo khu vực và / hoặc phạm vi giá không ?
[02] USER: tôi muốn nó là cheap và có chỗ đậu xe miễn phí .
[03] SYSTEM: có chín guesthouse hotels trong các khu vực various . bạn đang hy vọng ở khu vực nào của thị trấn ?
[04] USER: không có gì đặc biệt . tôi chỉ cần đặt phòng cho 6 người với tổng số 4 đêm bắt đầu từ sunday . tôi cũng muốn số tham chiếu , xin vui lòng .
[05] SYSTEM: chắc chắn rồi ! chúng tôi sẽ đặt bạn ở hotels với 4 sao . đặt phòng sẽ dành cho 6 người cho 4 đêm . bạn muốn đặt bao nhiêu phòng ?
[06] USER: tốt , đối với 6 , tôi cho rằng tôi sẽ cần ba phòng , cho 4 đêm , bắt đầu từ sunday .
[07] SYSTEM: bạn đã đặt phòng tại alexander bed and breakfast , 517a coldham lane , cho 6 cho 4 đêm trong su

 baseline Dialogue System 

Rule-based NLU: tách slot/value từ câu user bằng regex + keyword

Rule-based DST: cập nhật “belief state” (state của hội thoại)

Rule-based Policy: thấy thiếu slot nào thì hỏi tiếp, đủ slot thì xác nhận/hoàn tất

Template NLG: câu trả lời system sinh bằng mẫu

In [9]:
!python rule_tod_vi.py --dialog_id MUL0003.json --max_user_turns 30


=== RULE-BASED TOD (Level 2) ===
Dialog: MUL0003.json
Domains: ['hotel', 'restaurant']
SUCCESS: False

--- TRACE (first steps) ---

Step 0
User: tôi đang tìm một nơi để ở. nó phải là guesthouse và có wifi miễn phí .
NLU extracted: {'internet': 'yes', 'type': 'guesthouse'}
DST (current domain state): {'info': {'internet': 'yes', 'type': 'guesthouse', 'parking': 'yes', 'pricerange': 'cheap'}, 'book': {'people': 6, 'stay': 4, 'day': 'sunday'}}
Policy act: REQUEST(hotel.pricerange)
System: Bạn muốn mức giá rẻ / trung bình / đắt?

Step 1
User: tôi muốn nó là cheap và có chỗ đậu xe miễn phí .
NLU extracted: {'parking': 'yes', 'pricerange': 'cheap'}
DST (current domain state): {'info': {'internet': 'yes', 'type': 'guesthouse', 'parking': 'yes', 'pricerange': 'cheap'}, 'book': {'people': 6, 'stay': 4, 'day': 'sunday'}}
Policy act: REQUEST(hotel.people)
System: Bạn đặt cho mấy người?

Step 2
User: không có gì đặc biệt . tôi chỉ cần đặt phòng cho 6 người với tổng số 4 đêm bắt đầu từ sunday . tôi

In [15]:
!python run_rule_tod_demo.py --out rule_tod_demo_output.csv


Wrote: rule_tod_demo_output.csv


In [10]:
!python run_rule_tod_demo_eval.py --turns rule_tod_turns.csv --metrics rule_tod_metrics.csv


Wrote turns:   rule_tod_turns.csv
Wrote metrics: rule_tod_metrics.csv


Demo code này tương ứng phần User Simulation loop trong slide: user goal + user acts (intention), utterance generation, noise injection, rồi đánh giá bằng task success và slot/state accuracy.”
User Simulation architecture (Error/User/Reward + DM loop)

Slide mô tả: User simulator ↔ Dialogue Manager (DST + Policy) ↔ Backend, có thể có error model, có reward.
--------------------
User Model = phần user_reply() + sample_goal()
→ tạo goal + hành vi user (inform/confirm/deny+repair).

Error Model = noisy_slot_once + CONFUSABLE
→ cố tình cho user nói sai 1 slot (theater/time/mode/place) để tạo tình huống confirm + sửa lỗi.

Dialogue Manager (DM) = system_step() (policy “thiếu gì hỏi đó”) + update_state() (DST)
→ đúng loop: system request → user inform → DST update → policy chọn action.

Reward/Outcome: chưa trả reward per-turn kiểu RL, nhưng thay bằng evaluation outcome ở cuối: task_success, joint_goal_accuracy, num_rows.

In [16]:
!python  tod_simulator/run_tod_simulator_batch.py \
  --n 1000 \
  --turns tod_sim_turns_1000.csv \
  --metrics tod_sim_metrics_1000.csv \
  --summary tod_sim_summary.csv


Wrote turns:   tod_sim_turns_1000.csv
Wrote metrics: tod_sim_metrics_1000.csv
Wrote summary: tod_sim_summary.csv


In [12]:
!python tod_simulator/run_tod_simulator_batch_85.py \
  --n 1000 \
  --seed 7 \
  --turns tod_sim_turns_1000_85.csv \
  --metrics tod_sim_metrics_1000_85.csv \
  --summary tod_sim_summary_85.csv


Wrote turns:   tod_sim_turns_1000_85.csv
Wrote metrics: tod_sim_metrics_1000_85.csv
Wrote summary: tod_sim_summary_85.csv


In [17]:
!python tod_simulator/tod_hard_sim_and_train_v4.py --n 1000 --seed 7 \
  --p_nlu_drop 0.16 --p_nlu_confuse 0.09 --max_steps 8 \
  --p_refuse 0.11 --p_smalltalk 0.06 --p_oos 0.06 \
  --p_multi 0.55 --p_repair 0.66


/Users/admin/Documents/NLP/.venv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.8. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
DONE
